# Training and A/B test for Conversion agents - <font color='blue'> Proportion</font> of sale

In [1]:
import gym, recogym
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np
import pandas as pd
import pickle as pkl
pd.options.mode.chained_assignment = None 
# from inspect import getsource
from recogym.evaluate_agent_sale import verify_agents_sale, display_metrics, verify_agents_sale_extended
from tqdm import tqdm

# env_1_sale_args is a dictionary of default parameters (i.e. number of products)
from recogym import env_1_sale_args, Configuration
from recogym.agents.sale_agent import train_agents, train_timeagents
from recogym.envs.utils_sale import format_avg_result, avg_result, format_avg_result_extended, avg_result_extended 

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 0
env_1_sale_args['num_products'] = 10
env_1_sale_args['number_of_flips'] = 10 

num_products = env_1_sale_args['num_products']
print('Number of products =',num_products)
print('Number of flips =',env_1_sale_args['number_of_flips'])
nb_flips = env_1_sale_args['number_of_flips']

# You can overwrite environment arguments here:
env_1_sale_args['random_seed'] = 42
env_1_sale_args['mu_sale'] = False 

# env_1_sale_args['kappa'] = 0.5
print('Value of kappa =',env_1_sale_args['kappa'])

# Initialize the gym 
env = gym.make('reco-gym-sale-v1')
env.init_gym(env_1_sale_args)



Number of products = 10
Number of flips = 10
Value of kappa = 0.2


In [2]:
# Repository to save pickles
data_repo = 'data_conversion/'

## Train baseline agents

#### Settings

In [3]:
## Choose number of users for training and AB test
# Number of users for the training
# env_1_sale_args['num_users'] = 6 ##tochange !!
env_1_sale_args['num_users'] = 5000
num_users = env_1_sale_args['num_users']

# Number of users for the A/B test
# env_1_sale_args['num_users_AB'] = 7 ##tochange !!
env_1_sale_args['num_users_AB'] = 5000 
num_users_AB = env_1_sale_args['num_users_AB']

# Choose user features
from recogym.agents.sale_agent import CountViewsClicksFeatureProvider, CountViewsFeatureProvider, ShareViewsClicksFeatureProvider, ShareViewsFeatureProvider
vc_feature = CountViewsClicksFeatureProvider(env.config)
v_feature = CountViewsFeatureProvider(env.config)
vc_share_feature = ShareViewsClicksFeatureProvider(env.config)
v_share_feature = ShareViewsFeatureProvider(env.config)
features = {'vc':vc_feature,
           'v':v_feature,
           'vc_share':vc_share_feature,
           'v_share':v_share_feature}
feature_name = 'v_share'
feature = features[feature_name]

#### <font color='red'> Number of A/B tests</font>

In [4]:
# # Choose number of A/B tests
# num_AB_tests = 2 ##tochange !
num_AB_tests = 25 ##tochange !

#### Logs

In [5]:
agents={}
logs={}

############## Random agent
name_agent = 'rand'+str(nb_flips)
from recogym.agents import RandomAgent, random_args
random_agent = RandomAgent(Configuration(random_args))
agents[name_agent] = random_agent


try:
    logs[name_agent] = pkl.load(open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','rb'))
    print('--- Logs loaded---')
except: 
    print("--- Generate logs ---")
    logs[name_agent] = deepcopy(env).generate_logs(num_users)
    print(data_repo + 'data' + str(num_users) + name_agent + '.pkl')
    pkl.dump(logs[name_agent], open(data_repo + 'data' + str(num_users) + name_agent + '.pkl','wb'))

--- Logs loaded---


#### Training for the <font color='blue'> Proportion</font> of sale

##### No weights

In [6]:
name_extension = 'prop'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('timeagents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_timeagents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,linear_reg=True, repo = data_repo)

--- Trained agents loaded --- 


In [7]:
try:
    for i in range(num_AB_tests):
        name_extension = 'prop'+'_nb'+str(i)
        name = name = 'likelihood_saleclickprod_discount_allprop_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'prop'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [8]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.744 (0.403) %,1 (0),10.06 (0.00) %,6 (0),27.99 (8.46) %
1,Click,0.810 (0.058) %,1 (0),19.02 (4.23) %,6 (1),30.98 (4.23) %
2,PCS,0.514 (0.205) %,0 (0),13.04 (4.22) %,5 (5),27.98 (16.91) %
3,DPCS,0.816 (0.099) %,1 (0),16.03 (8.45) %,4 (3),22.01 (8.46) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.570 (0.020) %,5.05 (0.61) %,3.80 (0.32) %,4.10 (1.01) %,0.575 (0.050) pm,0.142 (0.028) pm,0.109 (0.049) pm
1,Click,0.590 (0.105) %,6.10 (0.08) %,5.06 (0.27) %,5.40 (0.05) %,0.718 (0.119) pm,0.125 (0.045) pm,0.082 (0.011) pm
2,PCS,0.728 (0.160) %,4.45 (0.36) %,3.04 (1.30) %,3.73 (0.62) %,0.654 (0.195) pm,0.190 (0.092) pm,0.101 (0.015) pm
3,DPCS,0.656 (0.108) %,4.75 (0.57) %,3.40 (1.40) %,3.75 (0.89) %,0.629 (0.178) pm,0.168 (0.080) pm,0.127 (0.020) pm


In [9]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
 Agent &              CTR & Att Sales &          Att CR &  Sales &               CR \\
\midrule
  Rand &  0.744 (0.403) \% &     1 (0) &  10.06 (0.00) \% &  6 (0) &   27.99 (8.46) \% \\
 Click &  0.810 (0.058) \% &     1 (0) &  19.02 (4.23) \% &  6 (1) &   30.98 (4.23) \% \\
   PCS &  0.514 (0.205) \% &     0 (0) &  13.04 (4.22) \% &  5 (5) &  27.98 (16.91) \% \\
  DPCS &  0.816 (0.099) \% &     1 (0) &  16.03 (8.45) \% &  4 (3) &   22.01 (8.46) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
 Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
  Rand &  0.570 (0.020) \% &  5.05 (0.61) \% &  3.80 (0.32) \% &  4.10 (1.01) \% &  0.575 (0.050) pm &  0.142 (0.028) pm &  0.109 (0.049) pm \\
 Click &  0.590 (0.105) \% &  6.10 (0.08) \% &  5.06 (0.27) \% &  5.40 (0.05) \% &  0.718 (0.119) pm &  0.125 (0.045) pm &  0.082 (0.011) pm \\
   PCS &  0

##### Weights

In [10]:
name_extension = 'propweights'
name_logging = name_agent
res_dict = {}
try :
    info, save_agents = pkl.load(open(data_repo+str('timeagents'+str(num_users)+name_logging+feature_name+name_extension+'.pkl'),'rb'))
    print('--- Trained agents loaded --- ')
except :
    print('--- Train agents ---')
    info, save_agents = train_timeagents(name_logging,logs,feature_name,features, num_users=num_users, 
                                     kronecker_features=True,linear_reg=True, weights = True, repo = data_repo)

--- Trained agents loaded --- 


In [11]:
try:
    for i in range(num_AB_tests):
        name_extension = 'propweights'+'_nb'+str(i)
        name = 'likelihood_saleclickprod_discount_allpropweights_full'
        agent_id = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension
        r = pkl.load(open(str(data_repo) +'res_'+agent_id+name+'.pkl',"rb"))
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r
    print('--- A/B test results loaded ---')
    
except:
    print(' --- Run A/B test --- ')
    for i in range(num_AB_tests):
        print("-------------- A/B test n°"+str(i)+'--------------')
        name_extension = 'propweights'+'_nb'+str(i)
        res=verify_agents_sale_extended(
            env,
            number_of_users=num_users_AB,
            agents={
                **agents, 
                **save_agents},
            name = name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension,
            seed = 0,
            same_env = False,
            repo = data_repo
        )

        r = deepcopy(res)
        for k in r.keys():
            if k not in ['CTR', 'Tot sales ATT', 'Share user with sale ATT', 'Tot sales', 'Share user with sale',
                         'True CTR','True PCS','True OS','True NCS']:
                r[k] = None

        res_dict[name_logging+str(num_users)+"_"+str(num_users_AB)+"_"+feature_name+name_extension] = r

--- A/B test results loaded ---


In [12]:
res_avg = avg_result_extended(res_dict)
(res_recap, res_recap_latex, 
 res_AB, res_AB_latex, 
 res_true, res_true_latex) = format_avg_result_extended(res_avg) #get dataframe & corresponding latex table

-- A/B test --


,Agent,CTR,Att Sales,Att CR,Sales,CR
0,Rand,0.591 (0.153) %,0 (0),10.06 (0.00) %,5 (1),27.99 (8.46) %
1,PCS,0.692 (0.168) %,1 (0),16.03 (0.00) %,3 (0),25.00 (4.23) %
2,DPCS,0.763 (0.318) %,1 (0),22.01 (0.00) %,8 (1),39.91 (8.40) %


-- True metrics --


,Agent,True CTR,True PCS,True OS,True NCS,NDPC,DPCSO,DPCSN
0,Rand,0.604 (0.018) %,4.46 (0.28) %,3.43 (0.18) %,3.41 (0.12) %,0.538 (0.017) pm,0.124 (0.008) pm,0.126 (0.015) pm
1,PCS,0.538 (0.148) %,4.59 (0.39) %,3.43 (0.66) %,3.51 (0.62) %,0.499 (0.179) pm,0.121 (0.006) pm,0.113 (0.008) pm
2,DPCS,0.528 (0.111) %,5.57 (0.41) %,3.79 (0.55) %,4.53 (0.49) %,0.584 (0.081) pm,0.190 (0.055) pm,0.111 (0.031) pm


In [13]:
print(res_AB_latex+'\n')
print(res_true_latex)

\begin{tabular}{llllll}
\toprule
Agent &              CTR & Att Sales &          Att CR &  Sales &              CR \\
\midrule
 Rand &  0.591 (0.153) \% &     0 (0) &  10.06 (0.00) \% &  5 (1) &  27.99 (8.46) \% \\
  PCS &  0.692 (0.168) \% &     1 (0) &  16.03 (0.00) \% &  3 (0) &  25.00 (4.23) \% \\
 DPCS &  0.763 (0.318) \% &     1 (0) &  22.01 (0.00) \% &  8 (1) &  39.91 (8.40) \% \\
\bottomrule
\end{tabular}


\begin{tabular}{llllllll}
\toprule
Agent &         True CTR &       True PCS &        True OS &       True NCS &              NDPC &             DPCSO &             DPCSN \\
\midrule
 Rand &  0.604 (0.018) \% &  4.46 (0.28) \% &  3.43 (0.18) \% &  3.41 (0.12) \% &  0.538 (0.017) pm &  0.124 (0.008) pm &  0.126 (0.015) pm \\
  PCS &  0.538 (0.148) \% &  4.59 (0.39) \% &  3.43 (0.66) \% &  3.51 (0.62) \% &  0.499 (0.179) pm &  0.121 (0.006) pm &  0.113 (0.008) pm \\
 DPCS &  0.528 (0.111) \% &  5.57 (0.41) \% &  3.79 (0.55) \% &  4.53 (0.49) \% &  0.584 (0.081) pm &  0.190 (0.